In [45]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# %inline matplotlib
# pd.set_option("display.max_rows", 60)
# pd.set_option("display.max_columns", 50)

# pd.options.display.max_rows
pd.set_option('display.max_colwidth',-1)

In [2]:
train = pd.read_csv('../data/raw/train.csv')
test = pd.read_csv('../data/raw/test.csv')
sample = pd.read_csv('../data/raw/sample_submission.csv')

In [5]:
print(train.shape)
train.head()

(108755, 3)


,plaintext_id,text,index
0,ID_d45641fc3,"come, who you are and what you would are out o...",104199
1,ID_2058482ae,For all in vain comes counsel to his ear.,79081
2,ID_5257a6819,Captain: With the next benefit o' the wind.,31234
3,ID_cd22c9b7c,"Smells wooingly here: no jutty, frieze,",56215
4,ID_dad4a8a96,"PORTIA: Which did incorporate and make us one,",47149


In [6]:
print(test.shape)
test.head()

(108755, 3)


,ciphertext_id,ciphertext,difficulty
0,ID_4a6fc1ea9,H2-t.'HzW$QOSvkPw v)4I1iSECKPX: P ktxjkp qemfl...,1
1,ID_9460d3380,Ah1QtVkLNcx7Q6mgvleEAqFIzyZoYkVtiKnd2Yy2t2GmoT...,4
2,ID_ac39fd360,BxBYt00fPc5zQqmls1mAFrVIzihsdFdoiKjdzZ22tWOnoT...,4
3,ID_d089e3234,AxFVtU0YMcl6R6mlvFKNFbVNziNpYkNtjKLX2Y+wsWShoT...,4
4,ID_55f57ffd0,11241 14155 31264 11448 35128 46135 39409 3201...,3


In [12]:
dif1 = test[test['difficulty'] == 1].copy()
dif1['ciphertext'].str.len()

0         100
5         100
19        100
20        100
23        100
         ... 
108723    100
108732    100
108736    100
108750    100
108751    100
Name: ciphertext, Length: 27158, dtype: int64

In [36]:
train[train['text'].str.len() < 100]

,plaintext_id,text,index
0,ID_d45641fc3,"come, who you are and what you would are out o...",104199
1,ID_2058482ae,For all in vain comes counsel to his ear.,79081
2,ID_5257a6819,Captain: With the next benefit o' the wind.,31234
3,ID_cd22c9b7c,"Smells wooingly here: no jutty, frieze,",56215
4,ID_dad4a8a96,"PORTIA: Which did incorporate and make us one,",47149
5,ID_4dd37fdb9,"My tears shall wipe away these bloody marks,",10692
6,ID_bbbaffe6b,I have done no harm. But I remember now,57607
7,ID_04fabf707,VALERIA: the yarn she spun in Ulysses' absence...,24925
8,ID_03be10ee9,"Second Murderer: O sir, it is better to be bri...",82259
9,ID_8b6c19ddc,In a most dear particular. He call'd me father:,27743


In [47]:
print(train[train['plaintext_id'] == 'ID_d45641fc3']['text'].str.len())
train[train['plaintext_id'] == 'ID_d45641fc3']['text']

0    50
Name: text, dtype: int64


0    come, who you are and what you would are out of my
Name: text, dtype: object

60
500
